# python语言与计算机科学引论
***
2017/12/2（2017秋季学期最后一节课）

上节课我们学习了面向对象编程中的基础：

* 将问题抽象为对象或对象与对象间的操作
* 对象的构造（```__init__```）
* 运算符重载

本节课继续讨论一些面向对象编程的概念，例如继承与多态。

先回顾一下上节课末尾实现的**有理数**类：

In [86]:
class Rat(object):
    
    def __init__(self, n, d):
        self.n = n
        self.d = d
    
    # self + a equals to q.add(a)
    def add(self, a):            
        new_numer = self.n*a.d + a.n*self.d
        new_denom = self.d*a.d
        return Rat(new_numer, new_denom)
    
    # 重载 + 运算
    def __add__(self, a):
        new_numer = self.n*a.d + a.n*self.d
        new_denom = self.d*a.d
        return Rat(new_numer, new_denom)
   
    # 重载 - 运算
    def __sub__(self, a):
        new_numer = self.n*a.d - a.n*self.d
        new_denom = self.d*a.d
        return Rat(new_numer, new_denom)
    
    def __eq__(self, a):
        return self.n*a.d == self.d*a.n
    '''
    __mul__    *
    __div__    /
    __eq__     ==
    '''
    
    # 改写实例输出
    def __str__(self):
        return '%d/%d' % (self.n, self.d)
    
    __repr__ = __str__

In [88]:
q1 = Rat(1, 2)
q2 = Rat(1, 3)
q3 = Rat(2, 4)
q1.add(q2)
q1==q2
def add(q1, q2):
    pass

False

### 访问限制

在Class内部，可以有**属性**和**方法**，而外部代码可以通过直接调用实例变量的方法来操作数据，这样，就隐藏了内部的复杂逻辑。

但是，从前面```Rat```类的定义来看，外部代码还是可以自由地修改一个实例的 ```n```、```d``` 属性：

In [36]:
q = Rat(1, 2)
q.n

1

In [37]:
q.n = 40     # 修改有理数实例q的分子
q

40/2

如果要让内部属性不被外部访问，可以把**属性的名称前加上两个下划线\__**，在Python中，实例的变量名如果以\__开头，就变成了一个**私有变量（private）**，只有内部可以访问，外部不能访问，所以，我们把```Rat```类改一改：

In [89]:
class Rat(object):
    
    def __init__(self, n, d):
        self.__n = n      # __n为私有变量
        self.__d = d      # __d为私有变量
        
    def __str__(self):
        return '%d/%d' % (self.__n, self.__d)  # 类内部可以访问私有变量
    
    __repr__ = __str__

改完后，对于外部代码来说，没什么变动，但是已经无法从外部访问 ```实例变量.__n``` 和 ```实例变量.__d``` 了：

In [90]:
q = Rat(1, 2)
q.__n

AttributeError: 'Rat' object has no attribute '__n'

这样就确保了外部代码不能随意修改对象内部的状态，这样通过访问限制的保护，代码更加**健壮**。

但是如果外部代码要获取 ```n``` 和 ```d``` 怎么办？可以给 ```Rat``` 类增加 ```numer``` 和 ```denom``` 这样的方法：

In [92]:
class Rat(object):
    
    def __init__(self, n, d):
        self.__n = n      # __n为私有变量
        self.__d = d      # __d为私有变量
    
    def numer(self):           # <---- 新增
        return self.__n
    
    def denom(self):           # <---- 新增
        return self.__d
    
    def __str__(self):
        return '%d/%d' % (self.__n, self.__d)
    __repr__ = __str__

In [93]:
q = Rat(1, 9)
print(q.numer(), q.denom())

1 9


如果又要允许外部代码修改```d```怎么办？可以再给```Rat```类增加 ```set_denom``` 方法：

In [95]:
class Rat(object):
    
    def __init__(self, n, d):
        self.__n = n      # __n为私有变量
        self.__d = d      # __d为私有变量
        
    def __str__(self):
        return '%d/%d' % (self.__n, self.__d)  # 类内部可以访问私有变量
    
    def numer(self):
        return self.__n
    
    def denom(self):
        return self.__d
    
    def set_denom(self, new_denom):                   # <---- 新增
        if isinstance(new_denom, int):  # 类型检查
            if new_denom:   # 除零检查
                self.__d = new_denom
            else:
                raise ValueError('use zero as denominator')
        else:
            raise ValueError('denominator must be integers')
    
    def __str__(self):
        return '%d/%d' % (self.__n, self.__d)
    __repr__ = __str__

In [96]:
q = Rat(1, 2)
q.set_denom(5)
q

1/5

你也许会问，原先那种直接通过 ```q.d = 5``` 也可以修改啊，为什么要定义一个方法大费周折？

因为在方法中，可以对参数做检查，避免传入无效的参数：

In [54]:
q.set_denom(0)

ValueError: use zero as denominator

### 继承与多态
在OOP程序设计中，当我们定义一个class的时候，可以从某个现有的class**继承**，新的class称为**子类（Subclass）**，而被继承的class称为**基类、父类或超类（Base class、Super class）**。

比如，我们已经编写了一个名为Animal的class，有一个run()方法可以直接打印：

In [97]:
class Animal():
    def run(self):
        print('Animal is running...')

当我们需要编写Dog和Cat类时，就可以直接从Animal类继承：

In [98]:
class Dog(Animal):
    pass

class Cat(Animal):
    pass

对于Dog来说，Animal就是它的父类，对于Animal来说，Dog就是它的子类。Cat和Dog类似。

继承有什么好处？最大的好处是子类获得了父类的全部功能。由于Animial实现了run()方法，因此，Dog和Cat作为它的子类，什么事也没干，就自动拥有了run()方法：

In [57]:
dog = Dog()
dog.run()

cat = Cat()
cat.run()

Animal is running...
Animal is running...


当然，也可以对子类增加一些方法，比如Dog类：

In [63]:
class Dog(Animal):

    def run(self):
        print('Dog is running...')

    def eat(self):
        print('Eating meat...')

继承的第二个好处需要我们对代码做一点改进。你看到了，无论是Dog还是Cat，它们run()的时候，显示的都是Animal is running...，符合逻辑的做法是分别显示Dog is running...和Cat is running...，因此，对Dog和Cat类改进如下：

In [99]:
class Dog(Animal):

    def run(self):
        print('Dog is running...')

class Cat(Animal):

    def run(self):
        print('Cat is running...')

再次运行，结果如下：

In [100]:
dog = Dog()
dog.run()

cat = Cat()
cat.run()

Dog is running...
Cat is running...


当子类和父类都存在相同的run()方法时，我们说，子类的run()覆盖了父类的run()，在代码运行的时候，总是会调用子类的run()。这样，我们就获得了继承的另一个好处：**多态**。

要理解什么是多态，我们首先要对数据类型再作一点说明。当我们定义一个class的时候，我们实际上就定义了一种数据类型。我们定义的数据类型和Python自带的数据类型，比如str、list、dict没什么两样：

In [104]:
a = list() # a是list类型 = []
b = Animal() # b是Animal类型
c = Dog() # c是Dog类型
c

判断一个变量是否是某个类型可以用isinstance()判断：

In [66]:
isinstance(a, list)

True

In [67]:
isinstance(b, Animal)

True

In [68]:
isinstance(c, Dog)

True

看来a、b、c确实对应着list、Animal、Dog这3种类型。

但是等等，试试：

In [69]:
isinstance(c, Animal)

True

看来c不仅仅是Dog，c还是Animal！

不过仔细想想，这是有道理的，因为Dog是从Animal继承下来的，当我们创建了一个Dog的实例c时，我们认为c的数据类型是Dog没错，但c同时也是Animal也没错，Dog本来就是Animal的一种！

所以，**在继承关系中，如果一个实例的数据类型是某个子类，那它的数据类型也可以被看做是父类**。但是，反过来就不行：

In [70]:
isinstance(b, Dog)

False

Dog可以看成Animal，但Animal不可以看成Dog。

要理解多态的好处，我们还需要再编写一个函数，这个函数接受一个Animal类型的变量：

In [109]:
def run_twice(animal):
    if isinstance(animal, Animal):
        animal.run()
        animal.run()
    else:
        raise ValueError('...')

当我们传入Animal的实例时，run_twice()就打印出：

In [106]:
run_twice(Animal())

Animal is running...
Animal is running...


当我们传入Dog的实例时，run_twice()就打印出：

In [107]:
run_twice(Dog())

Dog is running...
Dog is running...


当我们传入Cat的实例时，run_twice()就打印出：

In [75]:
run_twice(Cat())

Cat is running...
Cat is running...


看上去没啥意思，但是仔细想想，现在，如果我们再定义一个Tortoise类型，也从Animal派生：

In [108]:
class Tortoise(Animal):
    
    def run(self):
        print('Tortoise is running slowly...')

当我们调用run_twice()时，传入Tortoise的实例：

In [77]:
run_twice(Tortoise())

Tortoise is running slowly...
Tortoise is running slowly...


你会发现，新增一个Animal的子类，不必对run_twice()做任何修改，实际上，任何依赖Animal作为参数的函数或者方法都可以不加修改地正常运行，原因就在于多态。

多态的好处就是，当我们需要传入Dog、Cat、Tortoise……时，我们只需要接收Animal类型就可以了，因为Dog、Cat、Tortoise……都是Animal类型，然后，按照Animal类型进行操作即可。由于Animal类型有run()方法，因此，传入的任意类型，只要是Animal类或者子类，就会自动调用实际类型的run()方法，这就是多态的意思：

对于一个变量，我们只需要知道它是Animal类型，无需确切地知道它的子类型，就可以放心地调用run()方法，而具体调用的run()方法是作用在Animal、Dog、Cat还是Tortoise对象上，由运行时该对象的确切类型决定，这就是多态真正的威力：调用方只管调用，不管细节，而当我们新增一种Animal的子类时，只要确保run()方法编写正确，不用管原来的代码是如何调用的。这就是著名的“开闭”原则：

对扩展开放：允许新增Animal子类；

对修改封闭：不需要修改依赖Animal类型的run_twice()等函数。

继承还可以一级一级地继承下来，就好比从爷爷到爸爸、再到儿子这样的关系。而任何类，最终都可以追溯到根类object，这些继承关系看上去就像一颗倒着的树。继承还可以一级一级地继承下来，就好比从爷爷到爸爸、再到儿子这样的关系。而任何类，最终都可以追溯到根类object，这些继承关系看上去就像一颗倒着的树。比如如下的继承树：

                ┌───────────────┐
                │    object     │
                └───────────────┘
                        │
           ┌────────────┴────────────┐
           │                         │
           ▼                         ▼
    ┌─────────────┐           ┌─────────────┐
    │   Animal    │           │    Plant    │
    └─────────────┘           └─────────────┘
           │                         │
     ┌─────┴──────┐            ┌─────┴──────┐
     │            │            │            │
     ▼            ▼            ▼            ▼
┌─────────┐  ┌─────────┐  ┌─────────┐  ┌─────────┐
│   Dog   │  │   Cat   │  │  Tree   │  │ Flower  │
└─────────┘  └─────────┘  └─────────┘  └─────────┘



In [203]:
def gcd(a, b):
    r = b%a
    return a if r == 0 else gcd(r, a)

class Rat(object):
    
    def __init__(self, n=0, d=1):
        if isinstance(n, int) and isinstance(d, int):
            if d:
                if n:
                    q = abs(gcd(n, d))
                    n, d = (abs(n)//q, abs(d)//q) if n*d >= 0 else (-abs(n)//q, abs(d)//q)
                else:
                    d = 1
                self.n = n
                self.d = d
            else:
                raise ValueError('use zero as denominator')
        else:
            raise ValueError('denominator and numerator must be integers')
            
    def numer(self):
        return self.n
    
    def denom(self):
        return self.d
    
    # 重载 + 运算
    def __add__(self, a):
        new_numer = self.n*a.d + a.n*self.d
        new_denom = self.d*a.d
        return Rat(new_numer, new_denom)
   
    # 重载 - 运算
    def __sub__(self, a):
        new_numer = self.n*a.d - a.n*self.d
        new_denom = self.d*a.d
        return Rat(new_numer, new_denom)
    
    def __mul__(self, a):
        new_numer = self.n*a.n
        new_denom = self.d*a.d
        return Rat(new_numer, new_denom)
    
    def __truediv__(self, a):
        new_numer = self.n*a.d
        new_denom = self.d*a.n
        return Rat(new_numer, new_denom)
    
    def __eq__(self, a):
        return self.n*a.d == self.d*a.n
    
    def __str__(self):
        return '%d/%d' % (self.n, self.d) if self.d != 1 else str(self.n)
    __repr__ = __str__

In [201]:
from functools import reduce

def rsum(l):
    return reduce(lambda a, b: a+b, l)

class poly(object):

    def __init__(self, coef):

        def regular(c):
            return c if c[-1] != Rat() or len(c) == 1 else regular(c[:-1])

        self.coef = regular(coef)
        self.rank = len(self.coef)-1
    # len()
    def __len__(self):
        return self.rank

    def __add__(self, a):
        r = max(self.rank, a.rank)
        return poly([(self.coef[i] if i <= self.rank else Rat()) +
                     (a.coef[i] if i <= a.rank else Rat())
                     for i in range(r+1)])

    def __sub__(self, a):
        r = max(self.rank, a.rank)
        return poly([(self.coef[i] if i <= self.rank else Rat()) -
                     (a.coef[i] if i <= a.rank else Rat())
                     for i in range(r + 1)])

    def __mul__(self, a):
        return poly([rsum([self.coef[j] * a.coef[i-j] 
                           for j in range(i+1)
                           if 0 <= j <= self.rank and 0 <= i-j <= a.rank])
                     for i in range(self.rank+a.rank+1)])
    
    def __truediv__(self, a):
        def div_iter(p):
            rank = p.rank-a.rank
            if rank < 0:
                return poly([Rat()])
            term = poly([p.coef[-1]/a.coef[-1] if i == rank else Rat()
                         for i in range(rank+1)])
            # print(term)
            return term + div_iter(p-a*term)
        return div_iter(self)
            
    def __repr__(self):
        return str(self.coef)

In [166]:
rsum([Rat(), Rat(1)])

1

In [206]:
poly([Rat(i) for i in range(5)]) / poly([Rat(), Rat(1), Rat(3)])

[13/27, 5/9, 4/3]